In [4]:
import pandas as pd
from os import path
import json

In [5]:
DATA_DIR = "../../data"

In [2]:
seeds = [4, 18, 27, 36, 42]

In [6]:
split_results = {}
for seed in seeds:
    results_a = pd.read_csv(path.join(
        DATA_DIR, 
        "evaluations/protein_bert", 
        f"{str(seed)}a", 
        "all.csv"
    ), sep="\t", header=None)
    results_b = pd.read_csv(path.join(
        DATA_DIR, 
        "evaluations/protein_bert", 
        f"{str(seed)}b", 
        "all.csv"
    ), sep="\t", header=None)
    results_a.columns = ["model_name", "patience_stop", "patience_lr", "lr", "F1", "MCC", "Acc", "Precision", "Recall", "AUC"]
    results_b.columns = ["model_name", "patience_stop", "patience_lr", "lr", "F1", "MCC", "Acc", "Precision", "Recall", "AUC"]
    
    split_results[str(seed) + "a"] = results_a    
    split_results[str(seed) + "b"] = results_b

FileNotFoundError: [Errno 2] No such file or directory: '../../data/evaluations/protein_bert/36a/all.csv'

In [7]:
df_list = [value for key, value in split_results.items()]
# Concatenate results for all data splits
df_concat = pd.concat(df_list)

In [8]:
df_concat

,model_name,data,preprocessing,F1,MCC,Acc,Precision,Recall,AUC
0,kNN,integer_encoded,scaling,0.259740,0.132707,0.738532,0.389610,0.194805,0.550403
1,logistic_regression,integer_encoded,scaling,0.306785,0.067509,0.640673,0.281081,0.337662,0.535831
2,random_forest,integer_encoded,scaling,0.305455,0.125359,0.707951,0.347107,0.272727,0.557364
3,kNN,pybiomed,scaling,0.403974,0.225214,0.724771,0.412162,0.396104,0.611052
4,logistic_regression,pybiomed,scaling,0.458824,0.273214,0.718654,0.419355,0.506494,0.645247
...,...,...,...,...,...,...,...,...,...
37,logistic_regression,onehot,scaling,0.427673,0.237526,0.712025,0.400000,0.459459,0.624358
38,random_forest,onehot,scaling,0.444976,0.224873,0.632911,0.344444,0.628378,0.631338
39,multilayer_perceptron,onehot,scaling,0.360153,0.200262,0.735759,0.415929,0.317568,0.590602
40,SVM,onehot,scaling,0.411311,0.181260,0.637658,0.331950,0.540541,0.603948


In [9]:
grouped = df_concat.groupby(["model_name", "data"])
df_means = grouped.mean()

In [10]:
# Means across the different data splits
df_means

F1       MCC       Acc  \
model_name            data                                            
SVM                   bert             0.507515  0.362930  0.738193   
                      integer_encoded  0.337540  0.134286  0.670378   
                      onehot           0.430088  0.257971  0.720027   
                      protparam        0.303986  0.010289  0.457847   
                      pybiomed         0.440407  0.284269  0.745837   
                      sapiens          0.440024  0.267177  0.668604   
                      seqvec           0.489386  0.340335  0.674513   
gradient_boosting     bert             0.278702  0.202104  0.789875   
                      integer_encoded  0.169952  0.083143  0.768211   
                      onehot           0.223478  0.133043  0.773448   
                      protparam        0.303578  0.259314  0.806938   
                      pybiomed         0.239182  0.179683  0.791823   
                      sapiens          0.136349  0.102079  0.788166   
                      seqvec           0.209354  0.162010  0.790814   
kNN                   bert             0.353010  0.179221  0.720113   
                      integer_encoded  0.306184  0.147960  0.733124   
                      onehot           0.351777  0.173411  0.711698   
                      protparam        0.245259  0.034584  0.665917   
                      pybiomed         0.342626  0.171387  0.724376   
                      sapiens          0.278531  0.105469  0.713181   
                      seqvec           0.353629  0.173495  0.711447   
logistic_regression   bert             0.511498  0.372874  0.770851   
                      integer_encoded  0.315693  0.112783  0.674701   
                      onehot           0.445289  0.288201  0.748644   
                      protparam        0.364121  0.139984  0.592097   
                      pybiomed         0.467703  0.312537  0.742092   
                      sapiens          0.440547  0.268239  0.635459   
                      seqvec           0.510053  0.365731  0.735306   
multilayer_perceptron bert             0.464615  0.350077  0.801948   
                      integer_encoded  0.267335  0.135543  0.753369   
                      onehot           0.386076  0.259224  0.776552   
                      protparam        0.092959  0.016778  0.742383   
                      pybiomed         0.261821  0.205889  0.774164   
                      sapiens          0.281713  0.172617  0.765431   
                      seqvec           0.449182  0.324202  0.789333   
random_forest         bert             0.445334  0.316399  0.786602   
                      integer_encoded  0.349876  0.161722  0.696222   
                      onehot           0.421371  0.243352  0.703104   
                      protparam        0.461603  0.306881  0.744443   
                      pybiomed         0.207997  0.215087  0.805823   
                      sapiens          0.282824  0.189426  0.785031   
                      seqvec           0.465558  0.314144  0.756662   

                                       Precision    Recall       AUC  
model_name            data                                            
SVM                   bert              0.412027  0.667665  0.711918  
                      integer_encoded   0.288012  0.415756  0.576004  
                      onehot            0.371060  0.516170  0.643586  
                      protparam         0.205467  0.588839  0.505794  
                      pybiomed          0.400889  0.497485  0.653914  
                      sapiens           0.331980  0.655419  0.662689  
                      seqvec            0.358976  0.770563  0.709496  
gradient_boosting     bert              0.465092  0.203915  0.572146  
                      integer_encoded   0.319881  0.122160  0.528328  
                      onehot            0.380048  0.163042  0.546774  
                      protparam         0.573935  0.210143  0.585256